In [1]:
from osgeo import gdal

In [ ]:
gdal.Polygonize

In [ ]:
from osgeo import gdal, ogr
import os

def raster_to_polygon(raster_path, output_path, field_name="value"):
    # Open the source raster
    src_ds = gdal.Open(raster_path)
    if src_ds is None:
        raise FileNotFoundError(f"Could not open {raster_path}")
    
    # Get the first raster band
    band = src_ds.GetRasterBand(1)

    # Create output vector dataset (GeoJSON or Shapefile)
    driver_name = "GeoJSON" if output_path.lower().endswith(".geojson") else "ESRI Shapefile"
    driver = ogr.GetDriverByName(driver_name)
    
    # Remove output file if it already exists
    if os.path.exists(output_path):
        driver.DeleteDataSource(output_path)
    
    # Create new vector data source
    out_ds = driver.CreateDataSource(output_path)
    srs = None
    
    # Try to get spatial reference from raster
    if src_ds.GetProjection():
        srs = ogr.osr.SpatialReference(wkt=src_ds.GetProjection())
    
    # Create layer
    out_layer = out_ds.CreateLayer("polygons", srs=srs)
    
    # Create a new field in the output for raster values
    field_defn = ogr.FieldDefn(field_name, ogr.OFTInteger)
    out_layer.CreateField(field_defn)
    
    # Perform polygonization
    gdal.Polygonize(band, None, out_layer, 0, [], callback=None)
    
    # Cleanup
    out_ds = None
    src_ds = None
    print(f"✅ Polygonization complete: {output_path}")

if __name__ == "__main__":
    raster_to_polygon("input.tif", "output.geojson")


In [2]:
from osgeo import gdal, ogr
import numpy as np

# Open JPEG
ds = gdal.Open("raster.jpg")
band = ds.GetRasterBand(1)  # take the first channel (Red)

# Optionally, threshold to create a simple mask
arr = band.ReadAsArray()
mask = (arr > 128).astype(np.uint8)  # binary mask

# Create an in-memory raster from the mask
driver = gdal.GetDriverByName("MEM")
out_raster = driver.Create("", ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Byte)
out_raster.GetRasterBand(1).WriteArray(mask)

# Polygonize the mask
out_driver = ogr.GetDriverByName("GeoJSON")
out_ds = out_driver.CreateDataSource("output.geojson")
out_layer = out_ds.CreateLayer("polygons")
field_defn = ogr.FieldDefn("value", ogr.OFTInteger)
out_layer.CreateField(field_defn)

gdal.Polygonize(out_raster.GetRasterBand(1), None, out_layer, 0, [], callback=None)

out_ds = None
ds = None
print("Polygonization complete")


Polygonization complete


c:\Users\tanlezhan\OneDrive - Singapore University of Technology and Design\Documents\GitHub\Segmentation\.venv\Lib\site-packages\osgeo\gdal.py:330: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
